# solopy calibration example

In [2]:
import solopy
import logging
from ccdproc import ImageFileCollection
from pathlib import Path

In [3]:
# Observation date (=subdirectory name)
subdir = "2025_0803" 

# Data directory
WORKDIR = Path("../solo-data")

# Lv0 data directory
LV0DIR = WORKDIR/"Lv0"
LV0_subdir = LV0DIR / subdir

# Lv1 data directory
LV1DIR = WORKDIR/"Lv1"
LV1_subdir = LV1DIR / subdir

# Master calibration files directory
MASTERDIR = WORKDIR / "calibration_files"

# Log directory
LOGDIR = WORKDIR/"log"
LOGDIR.mkdir(parents=True, exist_ok=True)
fpath_log = LOGDIR/f'solopy_{subdir}.log' # general log file path
if fpath_log.exists():
    fpath_log.unlink()

### FitsLv0 (update header information)

In [4]:
allfits = ImageFileCollection(LV0_subdir, glob_include="*.fits")

In [5]:
lv0 = solopy.FitsLv0(log_file=fpath_log)

for fpath_fits in allfits.files_filtered(include_path=True):
    lv0.update_header(fpath_fits)

2025-11-29 01:22:59,317 [INFO] Updated FitsLv0 header: bias_001.fits
2025-11-29 01:22:59,395 [INFO] Updated FitsLv0 header: bias_002.fits
2025-11-29 01:22:59,471 [INFO] Updated FitsLv0 header: bias_003.fits
2025-11-29 01:22:59,550 [INFO] Updated FitsLv0 header: bias_004.fits
2025-11-29 01:22:59,627 [INFO] Updated FitsLv0 header: bias_005.fits
2025-11-29 01:22:59,703 [INFO] Updated FitsLv0 header: bias_006.fits
2025-11-29 01:22:59,781 [INFO] Updated FitsLv0 header: bias_007.fits
2025-11-29 01:22:59,853 [INFO] Updated FitsLv0 header: bias_008.fits
2025-11-29 01:22:59,931 [INFO] Updated FitsLv0 header: bias_009.fits
2025-11-29 01:23:00,006 [INFO] Updated FitsLv0 header: dark180_001.fits
2025-11-29 01:23:00,085 [INFO] Updated FitsLv0 header: dark180_002.fits
2025-11-29 01:23:00,157 [INFO] Updated FitsLv0 header: dark180_003.fits
2025-11-29 01:23:00,239 [INFO] Updated FitsLv0 header: dark180_004.fits
2025-11-29 01:23:00,317 [INFO] Updated FitsLv0 header: dark180_005.fits
2025-11-29 01:23:00

In [6]:
allfits.refresh()  # Refresh the file collection to reflect updated headers
allfits.summary.to_pandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   file      82 non-null     object 
 1   simple    82 non-null     bool   
 2   bitpix    82 non-null     int64  
 3   naxis     82 non-null     int64  
 4   naxis1    82 non-null     int64  
 5   naxis2    82 non-null     int64  
 6   lt        82 non-null     object 
 7   utc       82 non-null     object 
 8   jd        82 non-null     float64
 9   exptime   82 non-null     float64
 10  xbinning  82 non-null     int64  
 11  ybinning  82 non-null     int64  
 12  ccdtemp   82 non-null     float64
 13  ra        82 non-null     float64
 14  dec       82 non-null     float64
 15  alt       82 non-null     float64
 16  az        82 non-null     float64
 17  focus     82 non-null     int64  
 18  focallen  82 non-null     float64
 19  aptdia    82 non-null     float64
 20  pixsz     82 non-null     float64


### Combine Master Calibration Frame

In [7]:
bias_frame = allfits.files_filtered(imagetyp="BIAS", include_path=True)
dark_frame = allfits.files_filtered(imagetyp="DARK", include_path=True)
# sci_frame = allfits.files_filtered(imagetyp="LIGHT", include_path=True)

print(f"Number of bias frames: {len(bias_frame)}")
print(f"Number of dark frames: {len(dark_frame)}")

Number of bias frames: 9
Number of dark frames: 9


In [8]:
comb = solopy.CombMaster(log_file=fpath_log)

In [9]:
comb.comb_master_bias(bias_frame, MASTERDIR, outname="kl4040")

2025-11-29 01:23:12,954 [INFO] Starting master bias combination...
2025-11-29 01:23:12,955 [INFO] Loading 9 frames...
2025-11-29 01:23:13,078 [INFO] Combining 9 bias CCDData objects...
INFO:astropy:splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-11-29 01:23:24,772 [INFO] Master bias saved to kl4040.bias.comb.20250803.fits


PosixPath('../solo-data/calibration_files/kl4040.bias.comb.20250803.fits')

In [10]:
comb.comb_master_dark(dark_frame, MASTERDIR, outname="kl4040")

2025-11-29 01:23:24,786 [INFO] Starting master dark creation...
2025-11-29 01:23:24,786 [INFO] Loading 9 frames...
2025-11-29 01:23:24,911 [INFO] Searching for closest master bias to JD=2460891.0...
2025-11-29 01:23:24,957 [INFO] Using master bias: kl4040.bias.comb.20250803.fits
2025-11-29 01:23:25,212 [INFO] Combining 9 darks for exptime 180.0s...
INFO:astropy:splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-11-29 01:23:37,345 [INFO] Master dark saved to kl4040.dark.180s.comb.20250803.fits


[PosixPath('../solo-data/calibration_files/kl4040.dark.180s.comb.20250803.fits')]

### FitsLv1 (WCS update, preprocessing, zeropoint calculation)

In [8]:
allsci = allfits.files_filtered(imagetyp="LIGHT", include_path=True)
print("Number of science frames:", len(allsci))

Number of science frames: 266


In [9]:
lv1 = solopy.FitsLv1(log_file=fpath_log)

In [10]:
for fpath_sci in allsci:
    fpath_wcs = lv1.update_wcs(fpath_sci,
                               outdir=LV1_subdir,
                               cache_directory=WORKDIR/"astrometry_cache", 
                               return_fpath=True
                               )

2025-11-29 00:26:23,161 [INFO] WCS match: RA=331.93996, DEC=7.18443, scale=2.969
2025-11-29 00:26:23,467 [INFO] WCS updated: alt25az100_001_20250723055108.wcs.fits
2025-11-29 00:26:25,884 [INFO] WCS match: RA=331.93938, DEC=7.15865, scale=2.969
2025-11-29 00:26:25,910 [INFO] WCS updated: alt25az100_002_20250723055210.wcs.fits
2025-11-29 00:26:28,241 [INFO] WCS match: RA=331.93936, DEC=7.15865, scale=2.969
2025-11-29 00:26:28,268 [INFO] WCS updated: alt25az100_003_20250723055312.wcs.fits
2025-11-29 00:26:30,674 [INFO] WCS match: RA=331.93694, DEC=7.15710, scale=2.970
2025-11-29 00:26:30,700 [INFO] WCS updated: alt25az100_004_20250723055414.wcs.fits
2025-11-29 00:26:33,147 [INFO] WCS match: RA=331.93703, DEC=7.15700, scale=2.970
2025-11-29 00:26:33,173 [INFO] WCS updated: alt25az100_005_20250723055516.wcs.fits
2025-11-29 00:26:35,522 [INFO] WCS match: RA=312.15267, DEC=-22.10442, scale=2.967
2025-11-29 00:26:35,548 [INFO] WCS updated: alt25az150_001_20250723065438.wcs.fits
2025-11-29 00:

KeyboardInterrupt: 

In [ ]:
logger.info("--- Step 3:  WCS solution update ---")
lv1 = solopy.Lv1(log_file=fpath_log_lv1)

# Collect all Lv0 fits files
all_lv0_fits = ccdproc.ImageFileCollection(LV0_subdir, glob_exclude="*.bz2", glob_include="*.fits")
science_frames = all_lv0_fits.files_filtered(IMAGETYP="LIGHT", include_path=True)

TMPDIR = WORKDIR/"tmp"
TMPDIR.mkdir(parents=True, exist_ok=True)

if science_frames:
    for fpath_sci in science_frames:
        fpath_wcs = lv1.update_wcs(fpath_sci, TMPDIR, cache_directory=WORKDIR/"astrometry_cache", return_fpath=True)
        print(fpath_wcs)

2025-11-03 16:58:12,353 [INFO] --- Step 3:  WCS solution update ---
2025-11-03 16:58:16,896 [WARNING] Astrometry.net solver failed for longexp_001_20250719065814.fits: Solver.__init__() got an unexpected keyword argument 'log_level'
2025-11-03 16:58:16,896 [WARNING] Skipping center coord calculation for longexp_001_20250719065814.fits (no WCS).
2025-11-03 16:58:16,922 [INFO] WCS updated: longexp_001_20250719065814.wcs.fits


../solo-data/tmp/longexp_001_20250719065814.wcs.fits


2025-11-03 16:58:21,037 [WARNING] Astrometry.net solver failed for longexp_001_20250719070221.fits: Solver.__init__() got an unexpected keyword argument 'log_level'
2025-11-03 16:58:21,037 [WARNING] Skipping center coord calculation for longexp_001_20250719070221.fits (no WCS).
2025-11-03 16:58:21,055 [INFO] WCS updated: longexp_001_20250719070221.wcs.fits


../solo-data/tmp/longexp_001_20250719070221.wcs.fits


KeyboardInterrupt: 

2025-11-03 16:35:58,063 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-11-03 16:36:01,762 [INFO] downloading 3 files (169.19 MB)
2025-11-03 16:36:02,916 [INFO] downloading "http://data.astrometry.net/4100/index-4108.fits" to "/Users/bumhoo7/Desktop/solopy/notebook/astrometry_cache/4100/index-4108.fits"
2025-11-03 16:36:02,916 [INFO] downloading "http://data.astrometry.net/4100/index-4109.fits" to "/Users/bumhoo7/Desktop/solopy/notebook/astrometry_cache/4100/index-4109.fits"
2025-11-03 16:36:02,917 [INFO] downloading "http://data.astrometry.net/4100/index-4110.fits" to "/Users/bumhoo7/Desktop/solopy/notebook/astrometry_cache/4100/index-4110.fits"
2025-11-03 16:38:07,044 [INFO] loaded 3 index files
2025-11-03 16:38:07,046 [INFO] solve 1: start
2025-11-03 16:38:07,048 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:07,213 [INFO] solve 1: logodds=239.887, matches=42, conflicts=0, distractors=4, ra=270.196, dec=29.9716, scale=2.96297, index="4100/index-4108.fits"
2025-11-03 16:38:07,366 [INFO] solve 1: logodds=

../solo-data/tmp/longexp_001_20250719065814.wcs.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-11-03 16:38:13,381 [INFO] loaded 3 index files
2025-11-03 16:38:13,381 [INFO] solve 1: start
2025-11-03 16:38:13,383 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:13,555 [INFO] solve 1: logodds=238.681, matches=42, conflicts=0, distractors=4, ra=270.197, dec=29.972, scale=2.96288, index="4100/index-4108.fits"
2025-11-03 16:38:13,709 [INFO] solve 1: logodds=223.576, matches=42, conflicts=0, distractors=4, ra=270.196, dec=29.9683, scale=2.96347, index="4100/index-4108.fits"
2025-11-03 16:38:13,865 [INFO] solve 1: logodds=223.276, matches=42, conflicts=0, distractors=4, ra=270.195, dec=29.9691, scale=2.96366, index="4100/index-4108.fits"
2025-11-03 16:38:13,867 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:14,015 [INFO] solve 1: logodds=285.478, matches=41, conflicts=0, distractors=5, ra=270.194, dec=29.9708, scale=2.96246, index="4100/index-4109.fits"
2025-11-03 16:38:14,166 [INFO] solve 1:

...

2025-11-03 16:38:38,467 [INFO] loaded 3 index files
2025-11-03 16:38:38,468 [INFO] solve 1: start
2025-11-03 16:38:38,469 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:38,470 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:38,471 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4110.fits" (3 / 3)
2025-11-03 16:38:38,472 [INFO] solve 1: slice=[25, 50) (2 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:38,473 [INFO] solve 1: slice=[25, 50) (2 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:38,474 [INFO] solve 1: slice=[25, 50) (2 / 2), index="4100/index-4110.fits" (3 / 3)
2025-11-03 16:38:38,478 [WARNING] No WCS solution found for skyflat_001_20250719073146.fits.
2025-11-03 16:38:38,478 [WARNING] No WCS solution found for skyflat_001_20250719073146.fits.
2025-11-03 16:38:38,479 [WARNING] Skipping center coord calculation for skyflat_001_20250719073146.fits (no WCS).

../solo-data/tmp/skyflat_001_20250719073146.wcs.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-11-03 16:38:41,623 [INFO] loaded 3 index files
2025-11-03 16:38:41,624 [INFO] solve 1: start
2025-11-03 16:38:41,625 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4108.fits" (1 / 3)
2025-11-03 16:38:41,626 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4109.fits" (2 / 3)
2025-11-03 16:38:41,779 [INFO] solve 1: logodds=281.264, matches=43, conflicts=0, distractors=3, ra=270.125, dec=39.941, scale=2.9641, index="4100/index-4109.fits"
2025-11-03 16:38:41,935 [INFO] solve 1: logodds=285.128, matches=42, conflicts=0, distractors=4, ra=270.126, dec=39.9412, scale=2.9633, index="4100/index-4109.fits"
2025-11-03 16:38:42,087 [INFO] solve 1: logodds=272.733, matches=44, conflicts=0, distractors=2, ra=270.125, dec=39.9381, scale=2.97291, index="4100/index-4109.fits"
2025-11-03 16:38:42,089 [INFO] solve 1: slice=[0, 25) (1 / 2), index="4100/index-4110.fits" (3 / 3)
2025-11-03 16:38:42,237 [INFO] solve 1: logodds=324.462, matches=43, conflicts=0, distractors=3, ra=270.128

../solo-data/tmp/skyflat_001_20250719074006.wcs.fits
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


KeyboardInterrupt: 

In [ ]:



# Master bias
bias_files = all_lv0_fits.files_filtered(IMAGETYP="BIAS", include_path=True)
if bias_files:
    comb.make_mbias(bias_files, MASTERDIR)
    
# Master dark
dark_files = all_lv0_fits.files_filtered(IMAGETYP="DARK", include_path=True)
if dark_files:
    comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')